In [1]:
# Loading Libraries, Remember to run this cell!
library(tidyverse)
library(repr)
library(tidymodels)

options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

In [27]:
# Loading in the data
# Data read from https://drive.google.com/uc?export=download&id=1_MECmUXZuuILYeEOfonSGqodW6qVdhsS
player_stats <- read_csv("player_stats.csv")
colnames(player_stats) <- make.names(colnames(player_stats))
#   Remove unnecessary columns
player_stats_clean <- player_stats %>%
    select(Age:Peak.Elo.Rating, Retired, -Country, -Wikipedia, -Current.Rank,-Name, -Seasons, -Prize.Money, -Active, -Favorite.Surface, -Best.Elo.Rank) %>%
    na.omit() %>%
#   Mutating columns with as_factor()
    mutate(Dominate.Hand = as_factor(Plays)) %>%
    select(-Plays) %>%
    mutate(Backhand = as_factor(Backhand)) %>%
#   Cleaning up columns with string values
    mutate(Height = strtoi(str_remove(Height, " cm"))) %>%
    mutate(Year.Experience = 2020 - Turned.Pro) %>%
    mutate(Age = strtoi(substr(Age, 0, 2))) %>%
    mutate(Current.Elo.Rank = strtoi(substr(Current.Elo.Rank, 0, nchar(Current.Elo.Rank)-7))) %>%
    filter(!is.na(Turned.Pro)) %>%
    filter(is.na(Retired))

player_stats
player_stats_clean

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  .default = col_character(),
  X1 = col_double(),
  `Turned Pro` = col_double(),
  Seasons = col_double(),
  Titles = col_double(),
  `Best Season` = col_double(),
  Retired = col_double(),
  Masters = col_double(),
  `Grand Slams` = col_double(),
  `Davis Cups` = col_double(),
  `Team Cups` = col_double(),
  Olympics = col_double(),
  `Weeks at No. 1` = col_double(),
  `Tour Finals` = col_double()
)

See spec(...) for full column specifications.



X1,Age,Country,Plays,Wikipedia,Current.Rank,Best.Rank,Name,Backhand,Prize.Money,⋯,Facebook,Twitter,Nicknames,Grand.Slams,Davis.Cups,Web.Site,Team.Cups,Olympics,Weeks.at.No..1,Tour.Finals
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
0,26 (25-04-1993),Brazil,Right-handed,Wikipedia,378 (97),363 (04-11-2019),Oscar Jose Gutierrez,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,18 (22-12-2001),United Kingdom,Left-handed,Wikipedia,326 (119),316 (14-10-2019),Jack Draper,Two-handed,"$59,040",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,32 (03-11-1987),Slovakia,Right-handed,Wikipedia,178 (280),44 (14-01-2013),Lukas Lacko,Two-handed,"US$3,261,567",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
497,23 (14-03-1996),Netherlands,Left-handed,Wikipedia,495 (60),342 (05-08-2019),Gijs Brouwer,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
498,24 (17-05-1995),Ukraine,NA,Wikipedia,419 (81),419 (20-01-2020),Vladyslav Orlov,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
499,22 (26-03-1997),Tunisia,Left-handed,Wikipedia,451 (69),408 (24-12-2018),Aziz Dougaz,Two-handed,"$61,984",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


Age,Best.Rank,Backhand,Height,Turned.Pro,Current.Elo.Rank,Peak.Elo.Rating,Retired,Dominate.Hand,Year.Experience
<int>,<chr>,<fct>,<int>,<dbl>,<int>,<chr>,<dbl>,<fct>,<dbl>


In [23]:
set.seed(1234)
# Making the training and testing splits for each of the predictions
backhand_split <- initial_split(player_stats_clean, prop = .6, strata = Backhand)
backhand_training <- training(backhand_split)
backhand_testing <- testing(backhand_split)

hand_split <- initial_split(player_stats_clean, prop = .6, strata = Dominate.Hand)
hand_training <- training(hand_split)
hand_testing <- testing(hand_split)

# backhand_training
# backhand_testing
# hand_training
# hand_testing

Warning message:
“Too little data to stratify. Unstratified resampling will be used.”


ERROR: Error: Tibble columns must have compatible sizes.
* Size 2: Existing data.
* Size 0: Column `strata`.
[34mℹ[39m Only values of size one are recycled.


In [24]:
set.seed(1234)
# Creating the Recipes
backhand_recipe <- recipe(Backhand ~ Current.Elo.Rank + Year.Experience, data = backhand_training) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors())

hand_recipe <- recipe(Dominate.Hand ~ Current.Elo.Rank + Year.Experience, data = hand_training) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors())

# backhand_recipe
# hand_recipe

In [19]:
set.seed(1234)

# Creating the vfolds
backhand_vfold <- vfold_cv(backhand_training, v = 5, strata = Backhand)
hand_vfold <- vfold_cv(hand_training, v = 5, strata = Dominate.Hand)

In [20]:
set.seed(1234)
# Making the tune model to choose the best value of k
knn_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
        set_engine("kknn") %>%
        set_mode("classification")

In [21]:
set.seed(1234)
# Collecting metrics to determine best value of k
backhand_results <- workflow() %>%
    add_recipe(backhand_recipe) %>%
    add_model(knn_tune) %>%
    tune_grid(resamples = backhand_vfold, grid = 10) %>%
    collect_metrics

hand_results <- workflow() %>%
    add_recipe(hand_recipe) %>%
    add_model(knn_tune) %>%
    tune_grid(resamples = hand_vfold, grid = 10) %>%
    collect_metrics

backhand_results
hand_results

x Fold1: model 1/1 (predictions): Error: Problem with `mutate()` input `.row`.
[...

x Fold2: model 1/1 (predictions): Error: Problem with `mutate()` input `.row`.
[...

x Fold3: model 1/1 (predictions): Error: Problem with `mutate()` input `.row`.
[...

x Fold4: model 1/1 (predictions): Error: Problem with `mutate()` input `.row`.
[...

x Fold5: model 1/1 (predictions): Error: Problem with `mutate()` input `.row`.
[...

Warning message:
“All models failed in tune_grid(). See the `.notes` column.”


ERROR: Error: All of the models failed. See the .notes column.
